In [ ]:
!pip install git+https://github.com/jakeret/unet.git

In [ ]:
%load_ext tensorboard
import unet
from unet.datasets import oxford_iiit_pet
from tensorflow.keras import losses, metrics

In [ ]:
from typing import Optional, Union, Callable, List

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.optimizers import Adam

import unet.metrics
def finalize_model(model: Model,
                   loss: Optional[Union[Callable, str]]=losses.categorical_crossentropy,
                   optimizer: Optional= None,
                   metrics:Optional[List[Union[Callable,str]]]=None,
                   dice_coefficient: bool=True,
                   auc: bool=True,
                   mean_iou: bool=True,
                   **opt_kwargs):
    """
    Configures the model for training by setting, loss, optimzer, and tracked metrics
    :param model: the model to compile
    :param loss: the loss to be optimized. Defaults to `categorical_crossentropy`
    :param optimizer: the optimizer to use. Defaults to `Adam`
    :param metrics: List of metrics to track. Is extended by `crossentropy` and `accuracy`
    :param dice_coefficient: Flag if the dice coefficient metric should be tracked
    :param auc: Flag if the area under the curve metric should be tracked
    :param mean_iou: Flag if the mean over intersection over union metric should be tracked
    :param opt_kwargs: key word arguments passed to default optimizer (Adam), e.g. learning rate
    """

    if optimizer is None:
        optimizer = Adam(**opt_kwargs)

    if metrics is None:
        metrics = ['categorical_crossentropy',
                   'categorical_accuracy',
                   ]

    if mean_iou:
        metrics += [unet.metrics.mean_iou]

    if dice_coefficient:
        metrics += [unet.metrics.dice_coefficient]

    if auc:
        metrics += [tf.keras.metrics.AUC()]

    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics,
                  jit_compile=True
                  )

In [ ]:
LEARNING_RATE = 1e-3
unet_model = unet.build_model(*oxford_iiit_pet.IMAGE_SIZE,
                              channels=oxford_iiit_pet.channels,
                              num_classes=oxford_iiit_pet.classes,
                              layer_depth=5,
                              filters_root=64,
                              padding="same"
                              )

finalize_model(unet_model,
                    loss=losses.SparseCategoricalCrossentropy(),
                    metrics=[metrics.SparseCategoricalAccuracy()],
                    auc=False,
                    learning_rate=LEARNING_RATE)


In [ ]:
train_dataset, validation_dataset = oxford_iiit_pet.load_data()

In [ ]:
%tensorboard --logdir oxford_iiit_pet

In [ ]:
trainer = unet.Trainer(name="oxford_iiit_pet", checkpoint_callback=False)
trainer.fit(unet_model,
            train_dataset,
            validation_dataset,
            epochs=25,
            batch_size=32)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
rows = 10
fig, axs = plt.subplots(rows, 3, figsize=(8, 30))
for ax, (image, label) in zip(axs, train_dataset.take(rows).batch(1)):

  prediction = unet_model.predict(image)
  ax[0].matshow(image[0])
  ax[1].matshow(label[0, ..., 0], cmap="gray")
  ax[2].matshow(prediction[0].argmax(axis=-1), cmap="gray")